In [2]:
import pandas as pd
import glob



In [3]:

# Ruta donde están almacenados los archivos .csv
ruta_archivos = "Pruebas copia/Eyetracker Derecho/*.csv"  # Cambia esto a la ruta de tus archivos


In [10]:

# Lista para almacenar los dataframes de cada archivo
dataframes = []

# Leer y agregar cada archivo csv a la lista
for archivo in glob.glob(ruta_archivos):
    df = pd.read_csv(archivo,delimiter=';')
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
datos_completos = pd.concat(dataframes)

# Agrupar por 'coordenada x' y 'coordenada y', sumando la 'frecuencia de acierto'
resultado = datos_completos.groupby(['coordenada x [mm]', 'coordenada y [mm]'])['frecuencia de fallos'].sum().reset_index()

# Guardar el resultado en un nuevo archivo CSV
resultado.to_csv("frecuencia_acierto_total.csv", index=False)

print("Archivo 'frecuencia_acierto_total.csv' generado con éxito.")

Archivo 'frecuencia_acierto_total.csv' generado con éxito.
